In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 

In [2]:
import base64
import datetime
from urllib.parse import urlencode

import requests
import tqdm
import time

In [3]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token

    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
#         print(endpoint)
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
#         print(r.status_code)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    
    
    def temp(self, id_lst):
        for i in id_lst:
            print(self.get_resource(i, resource_type='tracks'))
            
    
    def get_genreSeeds(self):
        return self.get_resource("available-genre-seeds", resource_type='recommendations')
    
    def get_categories(self, _id):
        endpoint = f"https://api.spotify.com/v1/browse/categories/{_id}"
        print(endpoint)
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        print(r.status_code)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    
    def get_playlist_tracks(self, href):
        endpoint = href
        print(endpoint)
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        print(r.status_code)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_playlist(self, _id):
        return self.get_resource(_id, resource_type='playlists')
    
    def get_user_playlist(self, _id):
        endpoint = f"https://api.spotify.com/v1/users/{_id}/playlists"
        print(endpoint)
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        print(r.status_code)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def get_song(self, _id):
        return self.get_resource(lookup_id = _id, resource_type='tracks')
    
    def get_audiofeatures(self, _id):
        return self.get_resource(_id, resource_type='audio-features')
    
    def get_user(self, _id):
        return self.get_resource(_id, resource_type='users')
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, limit = 0, offset = 0, search_type='artist' ):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower(), "limit": limit, "offset": offset})
#         print(query_params)
        return self.base_search(query_params)

In [4]:
client_id = '47e0a0b995c44228ada0aba716861398'
client_secret = '457a089ac01441be87a0548c541308c0'

In [5]:
c_credentials = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=c_credentials)

In [6]:
spotify = SpotifyAPI(client_id, client_secret)

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/41R1UENL6yUDiRR2riptb7'},
 'followers': {'href': None, 'total': 1},
 'href': 'https://api.spotify.com/v1/playlists/41R1UENL6yUDiRR2riptb7',
 'id': '41R1UENL6yUDiRR2riptb7',
 'images': [{'height': 640,
   'url': 'https://mosaic.scdn.co/640/ab67616d0000b273722ee6054732c347e335cb35ab67616d0000b2737d4287bed60bfc5f13d96644ab67616d0000b273bef76218c65153abaed75e6eab67616d0000b273da6e3c1088d0a90bc96b848c',
   'width': 640},
  {'height': 300,
   'url': 'https://mosaic.scdn.co/300/ab67616d0000b273722ee6054732c347e335cb35ab67616d0000b2737d4287bed60bfc5f13d96644ab67616d0000b273bef76218c65153abaed75e6eab67616d0000b273da6e3c1088d0a90bc96b848c',
   'width': 300},
  {'height': 60,
   'url': 'https://mosaic.scdn.co/60/ab67616d0000b273722ee6054732c347e335cb35ab67616d0000b2737d4287bed60bfc5f13d96644ab67616d0000b273bef76218c65153abaed75e6eab67616d0000b273da6e3c1088d0a90bc96b848c',
   'width': 60}]

In [7]:
dictionary_playlist = spotify.get_user_playlist('smedjan')

https://api.spotify.com/v1/users/smedjan/playlists
200


In [8]:
all_playlists = dictionary_playlist["items"]

In [9]:
all_songs = []

In [10]:
for playlist in all_playlists:
    playlist_id = playlist['external_urls']['spotify'][-22:]
    get_track_href = spotify.get_playlist(playlist_id)['tracks']['href']
    all_tracks = spotify.get_playlist_tracks(get_track_href)
    lst = []
    for track in all_tracks['items']:
        try:
            song_id = track['track']['uri'][-22:]
#             print(song_id)
            lst.append(song_id)
        except:
            continue
    all_songs.append(lst)

https://api.spotify.com/v1/playlists/1CBGDKGM8kekBPfAG5jPZt/tracks?offset=0&limit=100
200
https://api.spotify.com/v1/playlists/3tT3E3Q4u5Xd0v3ySPLR1O/tracks?offset=0&limit=100
200
https://api.spotify.com/v1/playlists/41R1UENL6yUDiRR2riptb7/tracks?offset=0&limit=100
200
https://api.spotify.com/v1/playlists/6dSh5StaaE1XbIF5s7Lxng/tracks?offset=0&limit=100
200
https://api.spotify.com/v1/playlists/6hBpet9cW4cVgn96xBJ5Ay/tracks?offset=0&limit=100
200
https://api.spotify.com/v1/playlists/47s8Pj6MjV5UWYEaHIe9ZH/tracks?offset=0&limit=100
200
https://api.spotify.com/v1/playlists/13XT3kKAKOuhKV1IdgQxzf/tracks?offset=0&limit=100
200
https://api.spotify.com/v1/playlists/70lIZA6kTENcryaoGNIPaA/tracks?offset=0&limit=100
200
https://api.spotify.com/v1/playlists/6FTrJV6xPwAxEGNsIDG5Uj/tracks?offset=0&limit=100
200
https://api.spotify.com/v1/playlists/2tiFrJw4jgaQEDTGkNY4Lt/tracks?offset=0&limit=100
200
https://api.spotify.com/v1/playlists/0EDdjGp3CRtWBWesrDv45e/tracks?offset=0&limit=100
200
https://ap

In [11]:
import pandas as pd
import numpy as np

In [12]:
all_data = []

for i in range(len(all_songs)):
    file_name = 'user_playlist_' + (str)(i+1) + ".csv"
    lst = []
    for j in range(len(all_songs[i])):
        audiofeatures = spotify.get_audiofeatures(all_songs[i][j])
        print(i, j, len(audiofeatures))
        if len(audiofeatures) != 0:
            lst.append([all_songs[i][j], audiofeatures['acousticness'], audiofeatures['danceability'],
                        audiofeatures['energy'], audiofeatures['instrumentalness'], audiofeatures['key'], 
                        audiofeatures['liveness'],audiofeatures['loudness'], audiofeatures['mode'], 
                        audiofeatures['speechiness'], audiofeatures['tempo'], audiofeatures['time_signature'],
                        audiofeatures['valence']])
    df = pd.DataFrame(np.array(lst), columns = ['id', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 
                                      'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence'])

    df.to_csv(file_name)
            
    all_data.append(lst)

0 0 18
0 1 18
0 2 18
0 3 18
0 4 18
0 5 18
0 6 18
0 7 18
0 8 18
0 9 18
0 10 18
0 11 18
0 12 18
0 13 18
0 14 18
0 15 18
0 16 18
0 17 18
0 18 18
0 19 18
0 20 18
0 21 18
0 22 18
0 23 18
0 24 18
0 25 18
0 26 18
0 27 18
0 28 18
0 29 18
0 30 18
0 31 18
0 32 18
0 33 18
0 34 18
0 35 18
0 36 18
0 37 0
0 38 18
0 39 18
0 40 18
0 41 18
0 42 18
0 43 18
0 44 18
0 45 18
0 46 18
0 47 18
0 48 18
0 49 18
0 50 18
0 51 18
0 52 18
0 53 18
0 54 18
0 55 18
0 56 18
0 57 18
0 58 18
0 59 18
0 60 18
0 61 18
0 62 18
0 63 18
0 64 18
0 65 18
0 66 18
0 67 18
0 68 18
0 69 18
0 70 18
0 71 18
0 72 18
0 73 18
0 74 18
0 75 18
0 76 18
0 77 0
0 78 18
0 79 18
0 80 18
0 81 18
0 82 18
0 83 18
0 84 18
0 85 18
0 86 18
0 87 18
0 88 18
0 89 18
0 90 18
0 91 18
0 92 18
0 93 18
0 94 18
0 95 18
0 96 18
0 97 18
0 98 18
0 99 18
1 0 18
1 1 18
1 2 18
1 3 18
1 4 18
1 5 18
1 6 18
1 7 18
1 8 18
1 9 18
1 10 18
1 11 18
1 12 18
1 13 18
1 14 18
1 15 18
1 16 18
1 17 18
1 18 18
1 19 18
1 20 18
1 21 18
1 22 18
1 23 18
1 24 18
1 25 18
1 26 18
1 27 1

15 21 18
15 22 18
15 23 18
15 24 18
15 25 18
15 26 18
15 27 18
15 28 18
15 29 18
15 30 18
15 31 18
15 32 18
15 33 18
15 34 18
15 35 18
15 36 18
15 37 18
15 38 18
15 39 18
15 40 18
15 41 18
15 42 18
15 43 18
15 44 18
15 45 18
15 46 18
15 47 18
15 48 18
15 49 18
15 50 18
15 51 18
15 52 18
15 53 18
15 54 18
15 55 18
15 56 18
15 57 18
15 58 18
15 59 18
15 60 18
15 61 18
15 62 18
15 63 18
15 64 18
15 65 18
15 66 18
15 67 18
15 68 18
15 69 18
15 70 18
15 71 18
15 72 18
15 73 18
15 74 18
15 75 18
15 76 18
15 77 18
15 78 18
15 79 18
15 80 18
15 81 18
15 82 18
15 83 18
15 84 18
15 85 18
15 86 18
15 87 18
15 88 18
15 89 18
15 90 18
15 91 18
15 92 18
15 93 18
15 94 18
15 95 18
15 96 18
15 97 18
15 98 18
15 99 18
16 0 18
16 1 18
16 2 18
16 3 18
16 4 18
16 5 18
16 6 18
16 7 18
16 8 18
16 9 18
16 10 18
16 11 18
16 12 18
16 13 18
16 14 18
16 15 18
16 16 18
16 17 18
16 18 18
16 19 18
16 20 18
16 21 18
16 22 18
16 23 18
16 24 18
16 25 18
16 26 18
16 27 18
16 28 18
16 29 18
16 30 18
16 31 18
16 32 18
16

In [45]:
audiofeatures

{'danceability': 0.793,
 'energy': 0.839,
 'key': 8,
 'loudness': -12.658,
 'mode': 1,
 'speechiness': 0.0531,
 'acousticness': 0.303,
 'instrumentalness': 0.888,
 'liveness': 0.316,
 'valence': 0.439,
 'tempo': 130.775,
 'type': 'audio_features',
 'id': '78ZHugVlzEzw9WyBVPQl2s',
 'uri': 'spotify:track:78ZHugVlzEzw9WyBVPQl2s',
 'track_href': 'https://api.spotify.com/v1/tracks/78ZHugVlzEzw9WyBVPQl2s',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/78ZHugVlzEzw9WyBVPQl2s',
 'duration_ms': 326440,
 'time_signature': 4}

[[0,
  0.905,
  0.727,
  0.398,
  0.853,
  0,
  0.105,
  -11.804,
  1,
  0.0308,
  94.959,
  4,
  0.453],
 [1,
  0.303,
  0.793,
  0.839,
  0.888,
  8,
  0.316,
  -12.658,
  1,
  0.0531,
  130.775,
  4,
  0.439],
 [2,
  0.389,
  0.381,
  0.647,
  1.02e-05,
  2,
  0.972,
  -10.366,
  1,
  0.204,
  112.376,
  4,
  0.513],
 [3, 0.908, 0.395, 0.612, 0.363, 11, 0.13, -6.45, 1, 0.0371, 76.357, 4, 0.74],
 [4, 0.87, 0.626, 0.277, 0, 7, 0.0894, -9.767, 1, 0.0422, 124.505, 4, 0.383],
 [5,
  0.891,
  0.782,
  0.272,
  7.7e-06,
  0,
  0.0855,
  -9.537,
  1,
  0.0492,
  108.235,
  4,
  0.775],
 [6,
  0.75,
  0.373,
  0.58,
  4.05e-05,
  2,
  0.815,
  -9.594,
  1,
  0.0471,
  94.392,
  4,
  0.518],
 [7,
  0.808,
  0.633,
  0.418,
  0.00375,
  2,
  0.688,
  -8.528,
  1,
  0.0359,
  123.205,
  4,
  0.621],
 [8, 0.844, 0.496, 0.278, 0, 9, 0.13, -8.21, 1, 0.0537, 89.412, 4, 0.38],
 [9,
  0.827,
  0.538,
  0.492,
  3.29e-06,
  2,
  0.492,
  -7.837,
  1,
  0.0361,
  153.58,
  4,
  0.827],
 [10,
  0.948,
 

In [79]:
import pandas as pd


ValueError: 13 columns passed, passed data had 100 columns

In [46]:
len(dic1["items"])

20

In [21]:
playlist_id = dic1["items"][0]['external_urls']['spotify'][-22:]

In [27]:
get_track_href = spotify.get_playlist(playlist_id)['tracks']['href']

In [29]:
dic2 = spotify.get_playlist_tracks(get_track_href)

https://api.spotify.com/v1/playlists/1CBGDKGM8kekBPfAG5jPZt/tracks?offset=0&limit=100
200


In [36]:
song_id_1 = dic2['items'][0]['track']['uri'][-22:]

In [37]:
song_id_2 = dic2['items'][1]['track']['uri'][-22:]

In [47]:
len(dic2["items"])

100

In [38]:
song_id_1

'6SdkKrqkPUlzJf0AdRyxSt'

In [39]:
song_id_2

'78ZHugVlzEzw9WyBVPQl2s'

In [41]:
len(dic2['items'])

100

In [42]:
dic2['items'][0]

{'added_at': '2010-03-04T08:40:06Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/smedjan'},
  'href': 'https://api.spotify.com/v1/users/smedjan',
  'id': 'smedjan',
  'type': 'user',
  'uri': 'spotify:user:smedjan'},
 'is_local': False,
 'primary_color': None,
 'track': {'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0fvpn2k7FymYHxEx5U5FpP'},
     'href': 'https://api.spotify.com/v1/artists/0fvpn2k7FymYHxEx5U5FpP',
     'id': '0fvpn2k7FymYHxEx5U5FpP',
     'name': 'Gare Du Nord',
     'type': 'artist',
     'uri': 'spotify:artist:0fvpn2k7FymYHxEx5U5FpP'}],
   'available_markets': ['AD',
    'AE',
    'AG',
    'AL',
    'AM',
    'AO',
    'AR',
    'AT',
    'AU',
    'AZ',
    'BA',
    'BB',
    'BD',
    'BF',
    'BG',
    'BH',
    'BI',
    'BJ',
    'BN',
    'BO',
    'BR',
    'BT',
    'BW',
    'BY',
    'BZ',
    'CD',
    'CG',
    'CH',
    'CI',
    'CL',
    'CM',
    'CO',
  

In [43]:
spotify.get_song(song_id_2)

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6ULjGlnhcCwHIJigN47K5U'},
    'href': 'https://api.spotify.com/v1/artists/6ULjGlnhcCwHIJigN47K5U',
    'id': '6ULjGlnhcCwHIJigN47K5U',
    'name': 'AFX',
    'type': 'artist',
    'uri': 'spotify:artist:6ULjGlnhcCwHIJigN47K5U'}],
  'available_markets': [],
  'external_urls': {'spotify': 'https://open.spotify.com/album/6jhQBrBEdCxHCfnZmgeT7t'},
  'href': 'https://api.spotify.com/v1/albums/6jhQBrBEdCxHCfnZmgeT7t',
  'id': '6jhQBrBEdCxHCfnZmgeT7t',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b27333c4ba278bdf1f0a5c08690c',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e0233c4ba278bdf1f0a5c08690c',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d0000485133c4ba278bdf1f0a5c08690c',
    'width': 64}],
  'name': 'Analogue Bubblebath - EP',
  'release_date': '1994-02-22',
  'relea

In [44]:
audiofeatures = spotify.get_audiofeatures(song_id_2)

In [17]:
spotify.temp(['5vjLSffimiIP26QG5WcN2K', '6dwkFmLUz6hx2CZG2EqLol'])

{'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5D3muNJhYYunbRkh3FKgX0'}, 'href': 'https://api.spotify.com/v1/artists/5D3muNJhYYunbRkh3FKgX0', 'id': '5D3muNJhYYunbRkh3FKgX0', 'name': 'Chord Overstreet', 'type': 'artist', 'uri': 'spotify:artist:5D3muNJhYYunbRkh3FKgX0'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'LY', 'MA', 'MC', 'MD', 'ME', 'MG', 'MH', 'MK

In [42]:
ids[10]

'6dwkFmLUz6hx2CZG2EqLol'

In [31]:
genre_list = spotify.get_genreSeeds()['genres']

In [32]:
print(genre_list)

['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode', 'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post-dubstep', 'power-po

In [23]:
spotify.get_categories("happy")

{}

In [37]:
spotify.get_categories("dinner")

https://api.spotify.com/v1/browse/categories/dinner
404


{}

In [8]:
our_genre_list = ['acoustic', 'ambient', 'summer', 'blues', 'british', 'chill', 'classical', 'country', 'dance', 'death-metal',
                 'disco', 'drum-and-bass', 'dubstep', 'edm', 'electronic', 'emo', 'groove', 'guitar', 'happy', 'hip-hop', 
                 'jazz', 'k-pop', 'indian', 'party', 'pop', 'punk', 'reggae', 'rock', 'romance', 'sad', 'work-out', 'study']

In [10]:
ids = []
for i in range(len(our_genre_list)):
    print(i + 1)
    for j in range(20):
#         print("Song index:",i, "offset:", j)
        lst = spotify.search(query = "genre:" + our_genre_list[i], limit = 50, offset = j * 50, search_type = "track")
        it = lst['tracks']['items']
        for k in range(len(it)):
            ids.append(it[k]['uri'][14:])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [11]:
from collections import Counter

items = Counter(ids).keys()
print("No of unique items in the list are:", len(items))

No of unique items in the list are: 26938


In [12]:
song_data=[]

In [13]:
def getdata(song_id):
    audiofeatures = spotify.get_audiofeatures(song_id)
    print("searching")
    md = spotify.get_song(song_id)
    if md == {}:
        return "Get track failed"
    print("found")
    lst = [song_id ,md['name'], audiofeatures['acousticness'], audiofeatures['danceability'],
                    audiofeatures['energy'], audiofeatures['instrumentalness'], audiofeatures['key'], 
                    audiofeatures['liveness'],audiofeatures['loudness'], audiofeatures['mode'], 
                    audiofeatures['speechiness'], audiofeatures['tempo'], audiofeatures['time_signature'],
                    audiofeatures['valence'], md['album']['artists'][0]['name'], md['album']['name'], 
               md['album']['release_date'], md['duration_ms'], md['popularity']]
    return lst

In [14]:
print(getdata(ids[2]))


searching
Get track failed


In [ ]:
for i in range(len(ids)):
    print(i, end=" ")
    try:
        res = getdata(ids[i])
        song_data.append(res)
    except:
        pass

0 ['1EzrEOXmMH3G43AXT1y7pA', 0.559, 0.703, 0.444, 0, 11, 0.0973, -9.331, 1, 0.0417, 150.96, 4, 0.712]
1 ['5vjLSffimiIP26QG5WcN2K', 0.469, 0.618, 0.443, 0, 2, 0.0829, -9.681, 1, 0.0526, 119.949, 4, 0.167]
2 ['38YgZVHPWOWsKrsCXz6JyP', 0.241, 0.672, 0.623, 0, 9, 0.0848, -5.699, 1, 0.0297, 92.018, 4, 0.617]
3 ['7jIAttgQTpLDoNtykIQXjH', 0.316, 0.726, 0.537, 0, 0, 0.0707, -8.896, 1, 0.114, 96.889, 4, 0.882]
4 ['4E6cwWJWZw2zWf7VFbH7wf', 0.0208, 0.583, 0.786, 0, 2, 0.188, -3.142, 0, 0.0301, 123.055, 4, 0.573]
5 ['53QF56cjZA9RTuuMZDrSA6', 0.694, 0.483, 0.303, 0, 4, 0.115, -10.058, 1, 0.0429, 133.406, 3, 0.139]
6 ['0IktbUcnAGrvD03AWnz3Q8', 0.294, 0.625, 0.414, 0, 0, 0.151, -8.7, 1, 0.0369, 130.088, 4, 0.669]
7 ['6Uy6K3KdmUdAfelUp0SeXn', 0.00502, 0.551, 0.836, 2.1e-05, 10, 0.0425, -3.838, 0, 0.0524, 185.063, 4, 0.758]
8 ['1jyddn36UN4tVsJGtaJfem', 0.0304, 0.578, 0.727, 1.99e-05, 10, 0.172, -4.872, 1, 0.0332, 170.593, 4, 0.885]
9 ['5SuOikwiRyPMVoIQDJUgSV', 0.0322, 0.676, 0.461, 1.01e-06, 1, 0.358, 

79 ['3mDvi0k4LuCA7ViLf3Qb3O', 0.269, 0.759, 0.684, 0, 0, 0.13, -5.751, 1, 0.0315, 107.948, 4, 0.759]
80 ['2C3gh14oarzTtv3aY3HJ0m', 0.146, 0.625, 0.81, 2.72e-05, 10, 0.135, -5.995, 1, 0.033, 104.066, 4, 0.741]
81 ['0ifc4mq2mVNefgQFmtXQpt', 0.0653, 0.669, 0.854, 0, 4, 0.288, -5.899, 1, 0.0431, 120.035, 4, 0.706]
82 ['72SPsHheqN6J3sI520jT9E', 0.793, 0.681, 0.168, 0.000134, 7, 0.109, -13.397, 1, 0.0291, 128.615, 4, 0.29]
83 ['3bHhUEOTIbezeZ856R0BX5', 0.902, 0.561, 0.327, 0.000469, 11, 0.111, -10.835, 0, 0.0327, 161.858, 4, 0.298]
84 ['72R0X0h8YaxYNpegeoOl0M', 0.0758, 0.47, 0.801, 3.53e-05, 9, 0.218, -6.426, 1, 0.0352, 173.92, 4, 0.431]
85 ['0zANeX4R6uWb82gCQAguOD', 0.088, 0.62, 0.668, 0, 9, 0.362, -5.719, 1, 0.0305, 118.013, 4, 0.183]
86 ['6UOkF086zUgYXnqFxqUvEI', 0.843, 0.678, 0.277, 3.91e-06, 11, 0.104, -10.537, 1, 0.0602, 76.935, 4, 0.238]
87 ['3ktWkdvtG0CF0T2dEpek0H', 0.0966, 0.57, 0.662, 0, 8, 0.161, -3.872, 1, 0.0363, 94.948, 4, 0.856]
88 ['5IOmDUnTJwLQAuQnkPRCz5', 0.77, 0.59, 0.416,

157 ['2fTsxw433lT9uYnIH3ihDU', 0.959, 0.473, 0.334, 7.4e-05, 10, 0.28, -6.785, 1, 0.0371, 117.441, 4, 0.412]
158 ['4ngv7whSPSAwD8ZP3zMHsd', 0.844, 0.48, 0.618, 8.93e-05, 2, 0.325, -7.335, 1, 0.0501, 82.051, 4, 0.344]
159 ['2OTAulJdNdYoFQG2oqXtSB', 0.036, 0.636, 0.735, 1.23e-06, 6, 0.0915, -5.162, 1, 0.0318, 120.969, 4, 0.537]
160 ['0YPX6ztcWM4QTbR9tpnHNB', 0.0245, 0.616, 0.638, 3.59e-05, 7, 0.296, -7.981, 1, 0.0334, 89.989, 4, 0.507]
161 ['7wF4asYZw8cAHEljYd2Wid', 0.564, 0.592, 0.304, 0, 8, 0.107, -9.833, 1, 0.0298, 114.72, 3, 0.307]
162 ['4pAjn6PcQp81T2BKOzPOtn', 0.0158, 0.645, 0.766, 0.000164, 0, 0.151, -8.284, 0, 0.0872, 157.963, 4, 0.665]
163 ['1uSuY2qLCbx1QB7oPUVhFG', 0.601, 0.514, 0.569, 2.21e-06, 6, 0.218, -7.866, 0, 0.03, 142.597, 4, 0.264]
164 ['4MihgVRpD82EZoT61GW63w', 0.263, 0.619, 0.793, 0, 5, 0.0828, -5.964, 1, 0.0661, 177.991, 4, 0.892]
165 ['2pelUnMKr07JjJpilj9Yew', 0.119, 0.57, 0.711, 0, 9, 0.352, -4.155, 1, 0.0703, 163.117, 4, 0.648]
166 ['1Qo3taHLFSQpzLm5Sty7M4', 0.36

235 ['25fcj6d2W1l8DQL11Czdzb', 0.779, 0.354, 0.206, 0.00021, 0, 0.0959, -11.799, 1, 0.0365, 139.056, 4, 0.202]
236 ['6eDwkS4IHADqk06ssrqfUX', 0.2, 0.719, 0.444, 0, 7, 0.0982, -11.785, 1, 0.0952, 97.136, 4, 0.907]
237 ['1NoIZ1658UHpicKQ9rcWYh', 0.486, 0.623, 0.286, 4.44e-05, 9, 0.0884, -13.288, 1, 0.0991, 79.106, 4, 0.734]
238 ['1Zt5xYz9Fnsoa5rkbSPJWe', 0.809, 0.638, 0.513, 0.00103, 7, 0.134, -10.218, 1, 0.0328, 77.999, 4, 0.271]
239 ['2PIlBukQ6limukVR8Ubb5o', 0.737, 0.479, 0.541, 0, 0, 0.108, -9.862, 1, 0.0545, 85.994, 4, 0.321]
240 ['0c99BUd87HQfzgUUQqRyds', 0.927, 0.394, 0.249, 0, 5, 0.115, -10.484, 1, 0.0396, 79.549, 4, 0.329]
241 ['5PGYWv9Xp4IraLSDs8h1bY', 0.0756, 0.665, 0.635, 0, 5, 0.208, -5.302, 1, 0.0388, 113.046, 4, 0.299]
242 ['3ILmwMefYZoQh5Cf5jeuUQ', 0.076, 0.373, 0.914, 0, 0, 0.669, -4.185, 1, 0.0565, 168.21, 4, 0.56]
243 ['7kdJcH97qbvW7ubH6ukOTf', 0.895, 0.57, 0.296, 0.0353, 4, 0.115, -15.492, 1, 0.0349, 139.413, 4, 0.201]
244 ['2sCDzRlkLoeYpZX00i6iiq', 0.297, 0.702, 0.74

312 ['6owKuyHxUqidcAA6fPKSyy', 0.674, 0.705, 0.347, 0, 2, 0.12, -8.249, 1, 0.0301, 80.057, 4, 0.36]
313 ['5Oc0vLGWdEWeCqIU8zyELt', 0.706, 0.563, 0.542, 3.59e-05, 9, 0.355, -10.862, 1, 0.0962, 133.285, 4, 0.785]
314 ['3prWmTfS6HnvkKgezSQFJO', 0.0771, 0.602, 0.619, 0, 5, 0.0887, -8.194, 0, 0.0356, 107.428, 4, 0.228]
315 ['2Dzhwe90lhHXVU6lk7Nx0H', 0.777, 0.378, 0.52, 0.0888, 11, 0.0979, -10.965, 1, 0.0382, 73.744, 4, 0.357]
316 ['0iF2k99DJgFbPmsUQUFbtm', 0.0607, 0.791, 0.626, 0, 11, 0.803, -6.144, 1, 0.246, 96.96, 4, 0.707]
317 ['35RrDnREM1xxx4YBxDHikm', 0.13, 0.444, 0.771, 0, 0, 0.273, -4.607, 1, 0.0485, 148.028, 4, 0.45]
318 ['7k9GuJYLp2AzqokyEdwEw2', 0.426, 0.442, 0.632, 0.00419, 1, 0.0735, -6.77, 1, 0.0295, 78.899, 4, 0.196]
319 ['1XkKYN6tyLAYHjYkjNlbl5', 0.0833, 0.361, 0.436, 0, 11, 0.279, -5.834, 1, 0.0288, 67.353, 4, 0.334]
320 ['2Dms8axOrrjXETwVwtvBF5', 0.883, 0.554, 0.122, 0, 11, 0.107, -12.692, 1, 0.0614, 166.355, 4, 0.381]
321 ['5IGYPdHMSG4PLXDS9LDPJS', 0.956, 0.302, 0.0783, 9.

...

2823 ['2NkAoxQOr6MGdMB5JDTU81', 0.849, 0.479, 0.506, 2.69e-05, 11, 0.25, -8.175, 1, 0.0272, 136.662, 3, 0.83]
2824 ['5GWCndQ8ykzf6rYRvwrFcj', 0.838, 0.595, 0.279, 0.00605, 2, 0.0841, -12.794, 1, 0.0306, 92.345, 3, 0.212]
2825 ['6i9EhwGlQIUcZUAxrZ4Sdu', 0.131, 0.648, 0.614, 0.000611, 1, 0.0969, -9.358, 1, 0.0763, 94.035, 4, 0.339]
2826 ['6o21UfW2RHkzzleZGLfFpK', 0.24, 0.288, 0.62, 0, 2, 0.0882, -10.103, 1, 0.0428, 94.445, 4, 0.617]
2827 ['1CtKHwDqsH3ctcCCI18N0g', 0.61, 0.492, 0.928, 0.00259, 5, 0.368, -4.037, 1, 0.071, 194.078, 4, 0.92]
2828 ['6f5bVwzW72D5M8lZzH8Csg', 0.795, 0.736, 0.3, 1.96e-05, 4, 0.0701, -12.464, 0, 0.0759, 153.691, 4, 0.732]
2829 ['265iUOXC3O7IKs6OobU0Cx', 0.231, 0.533, 0.824, 5.19e-06, 5, 0.269, -4.2, 1, 0.101, 155.075, 4, 0.39]
2830 ['7v8YDCoM60xCPuSbXerImD', 0.743, 0.458, 0.164, 0, 0, 0.125, -23.883, 1, 0.0304, 103.32, 4, 0.82]
2831 ['1bSBbbH9KYzwsKosToLeAC', 0.773, 0.46, 0.3, 1.02e-05, 3, 0.248, -8.785, 0, 0.0962, 205.789, 3, 0.482]
2832 ['6pFQLoJefvBCTXtjO2dn4t

2899 ['3ut5AR80zVBxauB227w7ro', 0.000561, 0.484, 0.879, 0.438, 2, 0.528, -4.263, 1, 0.0579, 142.05, 4, 0.732]
2900 ['08JVaI77aIffN0wNjTOG4b', 0.514, 0.566, 0.521, 0.000288, 7, 0.116, -11.824, 1, 0.0321, 129.167, 4, 0.468]
2901 ['50J9zbu0UzOFnUVSjvecl7', 0.345, 0.649, 0.734, 0.00134, 8, 0.318, -5.454, 1, 0.0529, 114.651, 4, 0.693]
2902 ['7HXYoe65kmrPrJR1r0DGtW', 0.0523, 0.419, 0.613, 6.35e-05, 7, 0.361, -8.995, 1, 0.0316, 112.638, 4, 0.733]
2903 ['3kwtAEYzWabYBdSIfjoNmL', 0.204, 0.37, 0.755, 0.0266, 8, 0.663, -5.812, 1, 0.0552, 85.377, 4, 0.595]
2904 ['7lsnsM39mjdDKEbge48Ezx', 0.237, 0.286, 0.522, 0, 6, 0.32, -5.329, 1, 0.0312, 142.538, 4, 0.564]
2905 ['1yKah8BCP3Vgq2tPBOLKL0', 0.491, 0.321, 0.625, 5.08e-05, 2, 0.345, -5.219, 0, 0.0302, 107.338, 4, 0.777]
2906 ['65irrLqfCMRiO3p87P4C0D', 0.733, 0.579, 0.502, 0, 8, 0.281, -7.57, 1, 0.0513, 76.816, 4, 0.836]
2907 ['3QHA3SBpz2SDEpI1HuMT0e', 0.693, 0.434, 0.653, 0.0421, 1, 0.177, -6.969, 0, 0.0324, 105.09, 4, 0.625]
2908 ['3eYCJXTViFwl5mdtcT

2975 ['2OCoH12t6olzx9bT1PEC8F', 0.581, 0.632, 0.716, 2.48e-05, 10, 0.181, -12.387, 0, 0.0646, 164.851, 4, 0.935]
2976 ['3mNFoKDPM2hzPuJeZ9ziht', 0.0843, 0.542, 0.863, 0, 8, 0.0934, -5.006, 0, 0.0473, 119.996, 4, 0.431]
2977 ['618hiI74zBL8UVgAvfmkLj', 0.384, 0.462, 0.796, 0, 9, 0.296, -6.202, 1, 0.0322, 204.028, 4, 0.824]
2978 ['0xGSeBsG4V8Scc5YqpZQ66', 0.227, 0.4, 0.35, 0.0397, 8, 0.103, -11.401, 1, 0.0345, 83.389, 4, 0.328]
2979 ['5HgAZuHFAU5qLLMYuIQkgq', 0.03, 0.511, 0.766, 0.00752, 2, 0.221, -5.445, 0, 0.0473, 79.963, 4, 0.632]
2980 ['1soxeK1ZBX9SsSjK4j4ECa', 0.000173, 0.361, 0.799, 9.13e-05, 2, 0.496, -6.962, 0, 0.0392, 143.077, 4, 0.576]
2981 ['2jZPEeW2ynykb2GkVqpOkJ', 0.928, 0.638, 0.058, 0.079, 4, 0.113, -25.243, 1, 0.0388, 140.964, 4, 0.126]
2982 ['1S7QP5JYv6IuHGoQsOXzwR', 0.643, 0.601, 0.727, 0, 9, 0.28, -6.614, 1, 0.0394, 135.108, 4, 0.521]
2983 ['5Z10YWrfpP2UAUwmmHIR8y', 0.65, 0.537, 0.454, 0.000154, 9, 0.0852, -7.896, 1, 0.0551, 128.193, 4, 0.626]
2984 ['3XWiy9JEQ4iyqCzIqqd

3052 ['5v4OkFH0TJ6W1hCFxyn0px', 0.013, 0.612, 0.824, 0, 7, 0.288, -4.758, 1, 0.0478, 82.017, 4, 0.586]
3053 ['0pNeVovbiZHkulpGeOx1Gj', 0.198, 0.396, 0.338, 2.62e-06, 0, 0.138, -10.872, 1, 0.0304, 133.522, 4, 0.377]
3054 ['4oy98QoABnGKpvapDkI525', 0.95, 0.353, 0.446, 1.35e-05, 0, 0.132, -6.044, 0, 0.0285, 155.476, 4, 0.211]
3055 ['2DuPBbS5mIldXnh7Wum8Cy', 0.829, 0.339, 0.492, 0.00377, 1, 0.147, -10.661, 1, 0.0559, 151.865, 4, 0.102]
3056 ['4KROoGIaPaR1pBHPnR3bwC', 0.871, 0.642, 0.427, 0.0904, 3, 0.124, -8.471, 1, 0.0402, 82.344, 4, 0.404]
3057 ['3RNyGLgSvmVRZ7xKUp8Wgd', 0.66, 0.642, 0.65, 0, 4, 0.14, -8.567, 1, 0.0623, 164.421, 4, 0.973]
3058 ['3Am0IbOxmvlSXro7N5iSfZ', 0.336, 0.39, 0.502, 0.000138, 10, 0.0713, -12.277, 1, 0.178, 97.871, 4, 0.289]
3059 ['6nnacTL5on2aVsRhVDNUSo', 0.00176, 0.113, 0.951, 0.00718, 0, 0.334, -4.743, 0, 0.129, 200.024, 4, 0.139]
3060 ['1gFNm7cXfG1vSMcxPpSxec', 0.232, 0.818, 0.728, 0.0642, 10, 0.251, -8.331, 1, 0.0314, 113.059, 4, 0.975]
3061 ['6MOEQCtAWNSCmvg3

3128 ['61Q9oJNd9hJQFhSDh6Qlap', 0.000825, 0.315, 0.534, 0.108, 9, 0.101, -7.552, 0, 0.031, 116.891, 3, 0.283]
3129 ['4gWSTEBZxAvH1cKq2u8jfQ', 0.495, 0.605, 0.277, 1.87e-05, 1, 0.108, -11.333, 1, 0.142, 169.185, 4, 0.418]
3130 ['6im4Ks6SWUqwTGp7Cft1e7', 0.134, 0.536, 0.696, 3.17e-06, 4, 0.287, -6.818, 0, 0.0365, 117.179, 4, 0.683]
3131 ['0j3p1p06deJ7f9xmJ9yG22', 0.069, 0.48, 0.969, 0.00022, 9, 0.0608, -7.464, 1, 0.0854, 142.489, 4, 0.494]
3132 ['29b2b96jozyD9GPCkOrVLs', 0.12, 0.665, 0.782, 4.19e-06, 6, 0.125, -8.438, 0, 0.0307, 137.453, 4, 0.731]
3133 ['1DKyFVzIh1oa1fFnEmTkIl', 0.397, 0.641, 0.703, 0.00546, 9, 0.104, -10.434, 1, 0.0518, 129.118, 4, 0.722]
3134 ['0jRY4XO556pn1zHZAoY3ph', 0.0118, 0.608, 0.907, 0, 4, 0.186, -3.368, 1, 0.121, 89.994, 4, 0.818]
3135 ['7vvRkLPIvfjjmCIqNxBuEZ', 0.0515, 0.817, 0.484, 0.000213, 7, 0.0391, -11.659, 1, 0.143, 100.978, 4, 0.671]
3136 ['2QTDuJIGKUjR7E2Q6KupIh', 0.0249, 0.259, 0.583, 0.978, 2, 0.328, -9.489, 1, 0.0418, 111.153, 4, 0.24]
3137 ['3uliGw

In [48]:
spotify.get_audiofeatures(ids[0])

{'danceability': 0.703,
 'energy': 0.444,
 'key': 11,
 'loudness': -9.331,
 'mode': 1,
 'speechiness': 0.0417,
 'acousticness': 0.559,
 'instrumentalness': 0,
 'liveness': 0.0973,
 'valence': 0.712,
 'tempo': 150.96,
 'type': 'audio_features',
 'id': '1EzrEOXmMH3G43AXT1y7pA',
 'uri': 'spotify:track:1EzrEOXmMH3G43AXT1y7pA',
 'track_href': 'https://api.spotify.com/v1/tracks/1EzrEOXmMH3G43AXT1y7pA',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1EzrEOXmMH3G43AXT1y7pA',
 'duration_ms': 242947,
 'time_signature': 4}

In [1]:
len(['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode', 'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post-dubstep', 'power-pop', 'progressive-house', 'psych-rock', 'punk', 'punk-rock', 'r-n-b', 'rainy-day', 'reggae', 'reggaeton', 'road-trip', 'rock', 'rock-n-roll', 'rockabilly', 'romance', 'sad', 'salsa', 'samba', 'sertanejo', 'show-tunes', 'singer-songwriter', 'ska', 'sleep', 'songwriter', 'soul', 'soundtracks', 'spanish', 'study', 'summer', 'swedish', 'synth-pop', 'tango', 'techno', 'trance', 'trip-hop', 'turkish', 'work-out', 'world-music'])

126